# Standard CNN Architectures
* https://keras.io/applications/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [7]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.6


In [8]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [9]:
# for VGG, ResNet, and MobileNet
INPUT_SHAPE = (224, 224)

# for InceptionV3, InceptionResNetV2, Xception
# INPUT_SHAPE = (299, 299)

In [19]:
EPOCHS = 50

In [36]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 500

In [11]:
!rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [12]:
!ls -lh

total 7.0M
-rw-rw-r-- 1 ubuntu ubuntu 303K Sep 27 15:22 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rw-rw-r-- 1 ubuntu ubuntu 140K Sep 27 15:22 cnn-augmentation.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.6M Sep 27 15:22 cnn-comparing-all-models.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 380K Sep 27 15:27 cnn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.3M Sep 27 15:22 cnn-prediction.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  38K Sep 27 15:29 cnn-standard-architectures.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 198K Sep 27 15:22 cnn-train-augmented.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  48K Sep 27 15:22 imagenet-pretrained.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 495K Sep 27 15:22 london.jpg
drwxrwxr-x 3 ubuntu ubuntu 4.0K Sep 27 15:25 __MACOSX
-rw-rw-r-- 1 ubuntu ubuntu 127K Sep 27 15:22 Michigan-MSU-raschka.jpg
-rw-rw-r-- 1 ubuntu ubuntu 519K Sep 27 15:22 ml_intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 113K Sep 27 15:22 nn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu   63 Sep 27 15:22 README.html
drwxrwxr-x 8 ubuntu ubuntu 4.0K Sep 27 15:

In [13]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [14]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
X, y = load_data(original_dir, type=".ppm")

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((303, 224, 224, 3), (303, 6))

## VGG

In [32]:
from keras import applications
# applications.VGG16?
# model = applications.VGG16(include_top=False, weights='imagenet')
model = applications.VGG16(classes=6, weights=None)

In [33]:
# lacks dropout as this only exists during training

# model.summary()

In [34]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
# %time model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.3, callbacks=[tb_callback])

## MobileNet

In [16]:
from keras.applications.mobilenet import MobileNet

model = MobileNet(classes=6, weights=None)

In [21]:
# model.summary()

In [22]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
# %time model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.3, callbacks=[tb_callback])

Train on 212 samples, validate on 91 samples
Epoch 1/50
212/212 [==============================] - 5s - loss: 2.0239 - acc: 0.2170 - val_loss: 1.7894 - val_acc: 0.1429
Epoch 2/50
212/212 [==============================] - 3s - loss: 1.7915 - acc: 0.2642 - val_loss: 1.7966 - val_acc: 0.1429
Epoch 3/50
212/212 [==============================] - 3s - loss: 1.6676 - acc: 0.2877 - val_loss: 1.8036 - val_acc: 0.1429
Epoch 4/50
212/212 [==============================] - 3s - loss: 1.6491 - acc: 0.3821 - val_loss: 1.8192 - val_acc: 0.1429
Epoch 5/50
212/212 [==============================] - 3s - loss: 1.4930 - acc: 0.4104 - val_loss: 1.8490 - val_acc: 0.1429
Epoch 6/50
212/212 [==============================] - 3s - loss: 1.3754 - acc: 0.4623 - val_loss: 1.8935 - val_acc: 0.1429
Epoch 7/50
212/212 [==============================] - 3s - loss: 1.3231 - acc: 0.5189 - val_loss: 1.9417 - val_acc: 0.1429
Epoch 8/50
212/212 [==============================] - 3s - loss: 1.2213 - acc: 0.5849 - val_lo

In [24]:
# train_loss, train_accuracy = model.evaluate(X_train, y_train)
# train_loss, train_accuracy

303/303 [==============================] - 1s     


(5.0676892692893256, 0.1551155135183051)

In [25]:
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# test_loss, test_accuracy

76/76 [==============================] - 0s     


(5.7207725926449422, 0.10526315867900848)

## ResNet

In [26]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(classes=6, weights=None)

In [27]:
# model.summary()

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
!rm -rf tf_log

In [35]:
# https://github.com/fchollet/keras/issues/6014
# batch normalization seems to mess with accuracy when test data set is small, accuracy here is different from below
%time model.fit(X_train, y_train, epochs=100, validation_split=0.3, callbacks=[tb_callback], batch_size=BATCH_SIZE)

Train on 212 samples, validate on 91 samples
Epoch 1/100
212/212 [==============================] - 8s - loss: 0.2553 - acc: 0.9151 - val_loss: 2.0141 - val_acc: 0.5275
Epoch 2/100
212/212 [==============================] - 8s - loss: 0.1925 - acc: 0.9292 - val_loss: 1.9396 - val_acc: 0.4945
Epoch 3/100
212/212 [==============================] - 8s - loss: 0.4397 - acc: 0.8774 - val_loss: 3.4947 - val_acc: 0.2857
Epoch 4/100
212/212 [==============================] - 8s - loss: 0.3725 - acc: 0.8821 - val_loss: 1.9129 - val_acc: 0.4835
Epoch 5/100
212/212 [==============================] - 8s - loss: 0.2597 - acc: 0.9151 - val_loss: 2.9327 - val_acc: 0.4066
Epoch 6/100
212/212 [==============================] - 8s - loss: 0.1937 - acc: 0.9198 - val_loss: 3.7274 - val_acc: 0.4505
Epoch 7/100
212/212 [==============================] - 8s - loss: 0.1454 - acc: 0.9575 - val_loss: 1.9565 - val_acc: 0.5495
Epoch 8/100
212/212 [==============================] - 8s - loss: 0.2035 - acc: 0.9434 

212/212 [==============================] - 8s - loss: 0.0240 - acc: 0.9953 - val_loss: 2.5321 - val_acc: 0.6154
Epoch 67/100
212/212 [==============================] - 8s - loss: 0.0274 - acc: 0.9858 - val_loss: 2.0197 - val_acc: 0.7033
Epoch 68/100
212/212 [==============================] - 8s - loss: 0.0175 - acc: 0.9953 - val_loss: 1.9551 - val_acc: 0.6593
Epoch 69/100
212/212 [==============================] - 8s - loss: 0.0188 - acc: 0.9953 - val_loss: 3.2574 - val_acc: 0.5165
Epoch 70/100
212/212 [==============================] - 8s - loss: 0.1339 - acc: 0.9623 - val_loss: 2.7338 - val_acc: 0.5275
Epoch 71/100
212/212 [==============================] - 8s - loss: 0.1112 - acc: 0.9623 - val_loss: 2.6235 - val_acc: 0.5604
Epoch 72/100
212/212 [==============================] - 8s - loss: 0.0994 - acc: 0.9623 - val_loss: 2.3237 - val_acc: 0.6044
Epoch 73/100
212/212 [==============================] - 8s - loss: 0.0906 - acc: 0.9858 - val_loss: 2.7930 - val_acc: 0.5495
Epoch 74/100


In [41]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

303/303 [==============================] - 3s


(6.5429391860961914, 0.36303630471229553)

In [42]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

76/76 [==============================] - 0s


(6.9960803985595703, 0.26315790414810181)